In [ ]:
import sklearn
print(sklearn.__file__)
print(open(sklearn.__file__).readlines()[:2])
assert "yifrach was here" in open(sklearn.__file__).read(100), """This is not part of the exercise,
so if this assertion fails, then please send Yifrach the print ASAP"""

In [ ]:
import pandas as pd
import sklearn.datasets
import pathlib
import numpy as np
data_path = pathlib.Path(sklearn.datasets.__file__).parent /  'data/california_housing.csv'


In [ ]:
data = pd.read_csv(data_path)

In [ ]:
TARGET = "median_house_value"

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
numeric_data = data.drop(columns=["ocean_proximity"]).transform(
    pd.to_numeric, errors='coerce', axis=0)
# Rows without targets are useless for analysis
numeric_data = numeric_data.dropna(subset=[TARGET])


In [ ]:
data_train, data_test, _, _ = train_test_split(numeric_data, numeric_data.index, train_size=0.6, random_state=123)
data_train, data_test = data_train.copy(), data_test.copy()

In [ ]:
data_train["total_bedrooms"] = data_train["total_bedrooms"].fillna(data_train["total_bedrooms"].mean())
data_test["total_bedrooms"] = data_test["total_bedrooms"].fillna(data_train["total_bedrooms"].mean())


In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
results = {}
for layers_sizes in [(5,), (10,), (10, 10), (20, 10), (10, 20)]:
    reg = MLPRegressor(hidden_layer_sizes=layers_sizes, max_iter=30)
    reg.fit(data_train.drop(columns=[TARGET]), data_train[TARGET])
    
    for ocean_proximity, group in data[["ocean_proximity"]].groupby("ocean_proximity"):
        relevant_test_data = data_test[data_test.index.isin(group.index)]
        if not relevant_test_data.empty:
            score = reg.score(relevant_test_data.drop(columns=[TARGET]), relevant_test_data[TARGET])
            results.setdefault(ocean_proximity, {})[f"layers={layers_sizes}"] = score

In [ ]:
pd.DataFrame(results)[sorted(data["ocean_proximity"].unique())]

In [ ]:
from sklearn.cluster import k_means

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
scaler.fit(numeric_data.dropna())

In [ ]:
clusters1 = k_means(scaler.transform(numeric_data.dropna()),
                    n_clusters=5, random_state=123, max_iter=200)

In [ ]:
clusters2 = k_means(scaler.transform(numeric_data.dropna()), 
                    n_clusters=5, random_state=123, max_iter=200)

In [ ]:
# We used the same seed and same input - we expect the same result
assert (clusters1[1] == clusters2[1]).all()